In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from   scipy.stats import permutation_test
import seaborn

In [2]:
data_dir = os.path.join('..', 'data', 'derived')
df = pd.read_csv(os.path.join(data_dir, 'CONLIT_CharData_AP.csv.gz'), index_col='book_id')
df.head()

,char_id,char_count,inf_gender,gpe_places,num_gpe_places,nongpe_places,num_nongpe_places,all_places,num_all_places,gpe_sequences,...,char_rank,num_words,Category,Genre,Tokens,num_gpe_places_norm,num_nongpe_places_norm,num_gpe_places_norm_byCharacter,num_nongpe_places_norm_byCharacter,char_count_norm
book_id,,,,,,,,,,,,,,,,,,,,,
[Heist Society 2] Uncommon Criminals - Ally Carter,138,3238,she/her,"['Paraguay', 'Paraguay', 'London', 'New York C...",15,"['there', 'there', 'there', 'the museum', 'the...",103,"['there', 'there', 'there', 'Paraguay', 'Parag...",118,"['Paraguay', 'London', 'New York City', 'midto...",...,1,320000,FIC,YA,80392,0.000187,0.001281,0.004632,0.031810,0.040278
"2001_2011_Wilson,RobertCharles_TheChronoliths_SF",0,2742,she/her,"['Bangkok', 'Minneapolis / St. Paul', 'Chumpho...",22,"['our household', 'a gas station hawng nam', '...",122,"['our household', 'a gas station hawng nam', '...",143,"['Bangkok', 'Minneapolis / St. Paul', 'Chumpho...",...,1,320000,FIC,SF,103230,0.000213,0.001182,0.008023,0.044493,0.026562
"2001_Martel,Yann_LifeofPi_BS",0,4754,he/him/his,"['Canada', 'India', 'India', 'India', 'India',...",15,"['the north', 'the rich , noisy , functioning ...",142,"['Canada', 'India', 'the north', 'India', 'Ind...",157,"['Canada', 'India', 'Canada', 'India', 'Toront...",...,1,320000,FIC,BS,124445,0.000121,0.001141,0.003155,0.029870,0.038202
"2002_2011_Anderson,MT_Feed_SF",0,2862,she/her,"['Lonely', 'School', 'School', 'Switzerland', ...",5,"['the craters', 'a crummy hotel', 'here', 'her...",68,"['the craters', 'a crummy hotel', 'here', 'her...",72,"['Lonely', 'School', 'Switzerland', 'Io', 'Ame...",...,1,320000,FIC,SF,67075,0.000075,0.001014,0.001747,0.023760,0.042669
"2002_Baker,Jo_Offcomer_CT",120,4723,she/her,"['Belfast', 'Belfast', 'Belfast', 'Belfast', '...",11,"['the bath', 'the bath', 'the bath', 'it', 'th...",162,"['the bath', 'the bath', 'the bath', 'it', 'th...",172,"['Belfast', 'Conroys', 'October', 'Belfast', '...",...,1,320000,FIC,NYT,88042,0.000125,0.001840,0.002329,0.034300,0.053645


In [3]:
df.columns

Index(['char_id', 'char_count', 'inf_gender', 'gpe_places', 'num_gpe_places',
       'nongpe_places', 'num_nongpe_places', 'all_places', 'num_all_places',
       'gpe_sequences', 'dist_miles', 'char_rank', 'num_words', 'Category',
       'Genre', 'Tokens', 'num_gpe_places_norm', 'num_nongpe_places_norm',
       'num_gpe_places_norm_byCharacter', 'num_nongpe_places_norm_byCharacter',
       'char_count_norm'],
      dtype='object')

In [4]:
df.Category.value_counts()

Category
FIC    1934
NON     820
Name: count, dtype: int64

In [5]:
df.Genre.value_counts()

Genre
NYT     419
PW      258
BS      249
MY      234
MEM     229
SF      223
ROM     208
HIST    205
MIX     193
BIO     193
YA      177
MID     166
Name: count, dtype: int64

In [10]:
facet_by = ['inf_gender', 'Category', 'Genre']
cols_to_test = [
    'char_count',
    'num_gpe_places',
    'num_nongpe_places',
    'num_all_places',
    'dist_miles', 
    'Tokens', 
    'num_gpe_places_norm', 
    'num_nongpe_places_norm',
    'num_gpe_places_norm_byCharacter', 
    'num_nongpe_places_norm_byCharacter',
    'char_count_norm'
]

def significance_label(pvalue, levels=[0.05, 0.01, 0.001]):
    if pvalue < levels[2]:
        return '***'
    elif pvalue < levels[1]:
        return '**'
    elif pvalue < levels[0]:
        return '*'
    else: 
        return ''

def diff_means(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

def compare_facets(df, facets, cols, statistic=diff_means):
    '''Compare statistic in cols of df via permutation. Display results, return nothing.'''
    for facet in facets:
        print(f"\n==============\nFacet: {facet}\n==============")
        df = df.dropna(subset=[facet])
        for level in df[facet].unique():
            data = (df.loc[df[facet]==level, cols], df.loc[~(df[facet]==level), cols])
            level_mean = np.mean(data[0], axis=0)
            other_mean = np.mean(data[1], axis=0)
            output = pd.DataFrame(level_mean, columns=[level]).join(pd.DataFrame(other_mean, columns=['others']))
            res = permutation_test(data, statistic)
            output['diff'] = res.statistic
            output['p'] = res.pvalue
            output['sig'] = output['p'].apply(significance_label)
            display(output)

In [11]:
compare_facets(df, facet_by, cols_to_test, diff_means)


Facet: inf_gender


,she/her,others,diff,p,sig
char_count,3752.279775,3844.495428,-92.215652,0.4040,
num_gpe_places,16.356180,21.814954,-5.458775,0.0002,***
num_nongpe_places,122.757303,123.285637,-0.528334,0.8490,
num_all_places,138.620225,144.524476,-5.904251,0.0978,
dist_miles,56148.685011,79583.242665,-23434.557654,0.0002,***
Tokens,119516.989888,131900.656267,-12383.666379,0.0002,***
num_gpe_places_norm,0.000141,0.000170,-0.000029,0.0002,***
num_nongpe_places_norm,0.001127,0.001048,0.000079,0.0018,**
num_gpe_places_norm_byCharacter,0.005224,0.009569,-0.004345,0.0002,***
num_nongpe_places_norm_byCharacter,0.036178,0.037657,-0.001479,0.0362,*


,he/him/his,others,diff,p,sig
char_count,4012.726801,3475.705128,537.021673,0.0002,***
num_gpe_places,22.025360,16.663708,5.361652,0.0002,***
num_nongpe_places,127.342939,115.879684,11.463255,0.0002,***
num_all_places,148.785591,132.051282,16.734309,0.0002,***
dist_miles,80558.158976,57346.323422,23211.835554,0.0002,***
Tokens,133472.337752,118342.144970,15130.192782,0.0002,***
num_gpe_places_norm,0.000169,0.000146,0.000023,0.0002,***
num_nongpe_places_norm,0.001073,0.001074,-0.000001,0.9414,
num_gpe_places_norm_byCharacter,0.008396,0.007762,0.000634,0.1722,
num_nongpe_places_norm_byCharacter,0.036880,0.037687,-0.000807,0.2564,


,they/them/their,others,diff,p,sig
char_count,1490.612903,3924.422857,-2433.809954,0.0002,***
num_gpe_places,18.870968,20.103238,-1.232270,0.5390,
num_nongpe_places,66.516129,125.788190,-59.272061,0.0002,***
num_all_places,84.903226,145.339048,-60.435822,0.0002,***
dist_miles,65942.276537,72282.184946,-6339.908409,0.5660,
Tokens,109909.790323,128740.810286,-18831.019963,0.0016,**
num_gpe_places_norm,0.000183,0.000160,0.000024,0.0906,
num_nongpe_places_norm,0.000695,0.001091,-0.000396,0.0002,***
num_gpe_places_norm_byCharacter,0.025981,0.007321,0.018660,0.0002,***
num_nongpe_places_norm_byCharacter,0.048523,0.036642,0.011881,0.0002,***



Facet: Category


...

,PW,others,diff,p,sig
char_count,3177.618677,3880.336276,-702.717599,0.0002,***
num_gpe_places,14.801556,20.588684,-5.787127,0.0002,***
num_nongpe_places,121.428016,123.288523,-1.860508,0.7060,
num_all_places,135.782101,143.317416,-7.535315,0.1872,
dist_miles,52226.899827,74035.017864,-21808.118037,0.0022,**
Tokens,122261.315175,128472.023676,-6210.708501,0.1590,
num_gpe_places_norm,0.000134,0.000163,-0.000029,0.0012,**
num_nongpe_places_norm,0.001108,0.001070,0.000038,0.3532,
num_gpe_places_norm_byCharacter,0.005194,0.008469,-0.003275,0.0002,***
num_nongpe_places_norm_byCharacter,0.042319,0.036648,0.005672,0.0002,***


,MIX,others,diff,p,sig
char_count,1869.167539,3959.904222,-2090.736683,0.0002,***
num_gpe_places,25.450262,19.644253,5.806008,0.0008,***
num_nongpe_places,84.507853,125.997263,-41.489410,0.0002,***
num_all_places,109.403141,145.092651,-35.689509,0.0002,***
dist_miles,103312.422699,69657.898764,33654.523935,0.0008,***
Tokens,126088.335079,128026.023847,-1937.688768,0.7124,
num_gpe_places_norm,0.000204,0.000157,0.000047,0.0004,***
num_nongpe_places_norm,0.000745,0.001098,-0.000352,0.0002,***
num_gpe_places_norm_byCharacter,0.018654,0.007379,0.011275,0.0002,***
num_nongpe_places_norm_byCharacter,0.046469,0.036484,0.009985,0.0002,***


,MEM,others,diff,p,sig
char_count,4260.253275,3774.146032,486.107243,0.0136,*
num_gpe_places,37.877729,18.427381,19.450348,0.0002,***
num_nongpe_places,170.908297,118.771429,52.136868,0.0002,***
num_all_places,207.737991,136.694841,71.043150,0.0002,***
dist_miles,159239.594297,64068.139159,95171.455138,0.0002,***
Tokens,105611.161572,129916.065476,-24304.903904,0.0002,***
num_gpe_places_norm,0.000355,0.000143,0.000212,0.0002,***
num_nongpe_places_norm,0.001666,0.001019,0.000646,0.0002,***
num_gpe_places_norm_byCharacter,0.009739,0.008019,0.001720,0.0486,*
num_nongpe_places_norm_byCharacter,0.041595,0.036777,0.004819,0.0004,***


,MID,others,diff,p,sig
char_count,3103.355422,3860.351916,-756.996495,0.0006,***
num_gpe_places,7.024096,20.884630,-13.860534,0.0002,***
num_nongpe_places,94.024096,124.984127,-30.960031,0.0002,***
num_all_places,100.753012,145.303136,-44.550124,0.0002,***
dist_miles,16320.877678,75574.259419,-59253.381741,0.0002,***
Tokens,84156.487952,130702.076655,-46545.588703,0.0002,***
num_gpe_places_norm,0.000089,0.000165,-0.000076,0.0002,***
num_nongpe_places_norm,0.001243,0.001062,0.000181,0.0004,***
num_gpe_places_norm_byCharacter,0.002412,0.008532,-0.006120,0.0002,***
num_nongpe_places_norm_byCharacter,0.031853,0.037520,-0.005667,0.0002,***


,BIO,others,diff,p,sig
char_count,4291.549223,3778.629499,512.919724,0.0172,*
num_gpe_places,59.735751,17.050861,42.684891,0.0002,***
num_nongpe_places,175.652850,119.147496,56.505354,0.0002,***
num_all_places,233.875648,135.721831,98.153817,0.0002,***
dist_miles,252167.099482,58391.755702,193775.343780,0.0002,***
Tokens,196532.989637,122708.362285,73824.627352,0.0002,***
num_gpe_places_norm,0.000331,0.000148,0.000184,0.0002,***
num_nongpe_places_norm,0.001036,0.001076,-0.000040,0.3848,
num_gpe_places_norm_byCharacter,0.015934,0.007576,0.008359,0.0002,***
num_nongpe_places_norm_byCharacter,0.046214,0.036496,0.009718,0.0002,***
